# Elo score calculator

In [1]:
import pandas as pd
import numpy as np

Loads the tournaments.

In [2]:
tournaments = pd.read_excel('Data/MK8_Tournaments.xlsx', sheet_name='Scores')
tournaments = tournaments.copy()
tournaments['Date'] = tournaments['Date'].dt.strftime('%Y%m%d').astype(int)
tournaments['Elo'] = 0

tournaments.head()

,Date,Challenger,Round,Pts,Elo
0,20211006,Ilir,1,37,0
1,20211006,Alex,1,27,0
2,20211006,Sylvain,1,29,0
3,20211006,Sacha,1,54,0
4,20211006,Ilir,2,48,0


Creates the players dataframe with initial Elo scores of 1000.

In [3]:
players = pd.DataFrame(tournaments['Challenger'].unique(), columns=['Challenger'])

players['Elo'] = 1000
players['Nb_matches'] = 0
players['Last_update'] = tournaments['Date'].min()

players = players.set_index('Challenger')

players

,Elo,Nb_matches,Last_update
Challenger,,,
Ilir,1000,0,20211006
Alex,1000,0,20211006
Sylvain,1000,0,20211006
Sacha,1000,0,20211006
Julien,1000,0,20211006
Lev,1000,0,20211006
Olivier,1000,0,20211006
Antoine,1000,0,20211006
Rodolphe,1000,0,20211006


In [4]:
def prob(diff):
    return 1/(1 + (10**(-diff/400)))

prob(diff=147)

0.6997694032620582

In [5]:
results = pd.DataFrame(columns=['Date', 'Round', 'Winner', 'Loser', 'Count'])

In [6]:
def compute_elo(df, players) :

    score = df.copy()
    score['Rank'] = score['Pts'].rank(method='dense', ascending=False).astype(int)

    score = pd.merge(score[['Challenger', 'Rank', 'Date']], players['Elo'], on = 'Challenger')
    score.set_index('Challenger', inplace=True)

    # Creating an empty DataFrame for the 2D matrix of probabilities
    probs = pd.DataFrame(index=score.index, columns=score.index)

    # Creating an empty DataFrame for the 2D matrix of results (win/loss/draw)
    wins = pd.DataFrame(index=score.index, columns=score.index)

    game_result = pd.DataFrame(columns=['Winner', 'Loser', 'Count'])

    # Populating the matrices with the 
    for i in score.index:
        for j in score.index:
            # the probability of win is computed using the prob function of the Elo score difference between the players
            probs.at[i, j] = prob(score.at[i, 'Elo'] - score.at[j, 'Elo'])
            # If the player won, he gets a 1, if draw then 0.5 if loss then 0
            wins.at[i, j] = (1 - np.sign(score.at[i, 'Rank'] - score.at[j, 'Rank']))/2
            # Compute the winned matches 
            if i != j :
                game_result.loc[len(game_result)] = [i, j, wins.loc[i,j]]
    
    K = 32

    new_elo = pd.DataFrame((score['Elo'] + K * np.sum(wins - probs, axis=1)).astype('int64'), columns=['Elo'])

    # Updating the player dataframe
    # 1. Update 'Elo' score
    players.loc[new_elo.index, 'Elo'] = new_elo['Elo']

    # 2. Increment 'Nb_matches'
    players.loc[new_elo.index, 'Nb_matches'] += 1

    # 3. Update 'Last_update'
    players.loc[new_elo.index, 'Last_update'] = score['Date']

    df_elo = pd.merge(df[['Date', 'Challenger', 'Round', 'Pts']], new_elo, on='Challenger')
    df_elo.index = df.index

    return players, df_elo, game_result



Company origine
Left company

In [7]:
dates = tournaments['Date'].sort_values().unique().tolist()
tournament_date = dates[3]

tournament = tournaments[tournaments['Date'] == tournament_date]

early_game_limit = tournament['Round'].quantile(1/3)
middle_game_limit = tournament['Round'].quantile(2/3)

games = tournament['Round'].sort_values().unique().tolist()
game = games[4]

def compute_game_phase(game, early_game_limit, middle_game_limit):
    if game <= early_game_limit:
        return '1-Early'
    elif game <= middle_game_limit:
        return '2-Middle'
    else :
        return '3-Late'


df = tournament[tournament['Round'] == game]

# Compute game phase
df['Game_phase'] = compute_game_phase(game, early_game_limit, middle_game_limit)

# ranks the players using the column Pts
df['Rank'] = df['Pts'].rank(method='dense', ascending=False).astype(int)

df

/var/folders/rm/zd360c_n3394v5zq_gnyj9s00000gn/T/ipykernel_77677/1892295998.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Game_phase'] = compute_game_phase(game, early_game_limit, middle_game_limit)
/var/folders/rm/zd360c_n3394v5zq_gnyj9s00000gn/T/ipykernel_77677/1892295998.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rank'] = df['Pts'].rank(method='dense', ascending=False).astype(int)


,Date,Challenger,Round,Pts,Elo,Game_phase,Rank
176,20220207,Lev,5,51,0,2-Middle,1
177,20220207,Antoine,5,46,0,2-Middle,2
178,20220207,Julien,5,46,0,2-Middle,2
179,20220207,Ilir,5,38,0,2-Middle,3


In [ ]:
dates = tournaments['Date'].sort_values().unique().tolist()
dates

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [8]:
dates = tournaments['Date'].sort_values().unique().tolist()

for tournament_date in dates:
    tournament = tournaments[tournaments['Date'] == tournament_date]
    
    early_game_limit = tournament['Round'].quantile(1/3)
    middle_game_limit = tournament['Round'].quantile(2/3)

    games = tournament['Round'].sort_values().unique().tolist()

    for game in games:
        players, df_elo, game_result = compute_elo(df=tournament[tournament['Round'] == game], players=players)
        
        tournaments.loc[df_elo.index, 'Elo'] = df_elo['Elo']
        game_result['Date'] = tournament_date
        game_result['Round'] = game

        tournaments.loc[df_elo.index, 'Rank'] = df_elo['Pts'].rank(method='dense', ascending=False).astype(int)
        tournaments.loc[df_elo.index, 'Game_phase'] = compute_game_phase(game, early_game_limit, middle_game_limit)

        results = pd.concat([results, game_result], ignore_index=True)
    
    print('Elo score computed for tournament: ' + str(tournament_date))

players.sort_values('Elo', ascending=False)

/var/folders/rm/zd360c_n3394v5zq_gnyj9s00000gn/T/ipykernel_77677/849445898.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, game_result], ignore_index=True)


Elo score computed for tournament: 20211006
Elo score computed for tournament: 20211103
Elo score computed for tournament: 20211201
Elo score computed for tournament: 20220207
Elo score computed for tournament: 20220307
Elo score computed for tournament: 20220404
Elo score computed for tournament: 20220502
Elo score computed for tournament: 20220607
Elo score computed for tournament: 20220704
Elo score computed for tournament: 20220808
Elo score computed for tournament: 20220905
Elo score computed for tournament: 20221003
Elo score computed for tournament: 20221107
Elo score computed for tournament: 20221205
Elo score computed for tournament: 20221214
Elo score computed for tournament: 20230109
Elo score computed for tournament: 20230206
Elo score computed for tournament: 20230306
Elo score computed for tournament: 20230405
Elo score computed for tournament: 20230508
Elo score computed for tournament: 20230605
Elo score computed for tournament: 20230705
Elo score computed for tournamen

,Elo,Nb_matches,Last_update
Challenger,,,
Sacha,1523,149,20250303
Lev,1370,238,20250410
Sébastien,1342,62,20250303
Luca,1340,212,20250410
Romain,1275,68,20250410
Denes,1199,43,20250410
Ilir,1138,128,20250410
Martin,1136,79,20250410
Julien,1096,233,20250303


In [9]:
tournaments.head()

,Date,Challenger,Round,Pts,Elo,Rank,Game_phase
0,20211006,Ilir,1,37,1016,2.0,1-Early
1,20211006,Alex,1,27,952,4.0,1-Early
2,20211006,Sylvain,1,29,984,3.0,1-Early
3,20211006,Sacha,1,54,1048,1.0,1-Early
4,20211006,Ilir,2,48,1029,2.0,1-Early


In [10]:
tournaments.to_excel('Data/MK8_Tournaments_Elo.xlsx', sheet_name='Scores', index=False)

In [11]:
results.to_excel('Data/MK8_Tournaments_wins.xlsx', sheet_name='Scores', index=False)